In [1]:
import mediapipe as mp 
import cv2 as cv
import time
from cvzone.HandTrackingModule import HandDetector as htm
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import math
import numpy as np

In [4]:
import cv2 as cv
from cvzone.HandTrackingModule import HandDetector
import time

cap = cv.VideoCapture(0)
wCam, hCam = 640, 480
cap.set(3, wCam)
cap.set(4, hCam)

detector = HandDetector(detectionCon=0.8)


devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = interface.QueryInterface(IAudioEndpointVolume)
# volume.GetMute()
# volume.GetMasterVolumeLevel()
volRange = volume.GetVolumeRange()
minVol = volRange[0]
maxVol = volRange[1]
pTime = 0
vol = 0
volBar = 0
volPer =0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Flip the frame horizontally
    flip = cv.flip(frame, 1)

    hands, flip = detector.findHands(flip, flipType=False)

    if hands:
        lmList = hands[0]['lmList']
        if lmList:

            x1, y1 = lmList[4][0], lmList[4][1]
            x2, y2 = lmList[8][0], lmList[8][1]
            cx , cy = int((x1+x2)/2) , int((y1+y2)/2)

            cv.circle(flip, (x1, y1), 15, (255, 0, 233), cv.FILLED)
            cv.circle(flip, (x2, y2), 15, (255, 0, 233), cv.FILLED)
            cv.circle(flip, (cx, cy), 15, (255, 0, 233), cv.FILLED)
            
            cv.line(flip, (x1, y1), (x2, y2), (200, 0, 255), 3)
            
            length = math.hypot(x2-x1,y2-y1)
            print(length)
            #volRange -65 - 0 , need to convert 50 - 300 
            vol = np.interp(length,[20,240],[minVol , maxVol])
            volBar = np.interp(length,[20,200],[400 , 150])
            volPer = np.interp(length,[20,200],[0 , 100])
            volume.SetMasterVolumeLevel(vol, None)
            
            if length < 20:
                cv.circle(flip, (cx, cy), 15, (0, 255, 2), cv.FILLED)
                
            cv.rectangle(flip,(50,150),(85,400),(133,133,0),3)
            cv.rectangle(flip,(50,int(volBar)),(85,400),(133,133,0),cv.FILLED)
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime

    cv.putText(flip, f'Volume: {int(volPer)}', (10, 40), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

    cv.imshow("frame", flip)

    if cv.waitKey(1) & 0xFF == ord(' '):
        break
cap.release()
cv.destroyAllWindows()


167.6305461424021
172.40939649566667
186.93581786270923
192.92485583770693
197.69926656414282
198.38598740838526
199.9199839935968
200.8606482116395
201.8018830437417
202.08166665979377
201.4671189052943
202.07176942858692
203.06649157357302
201.2411488736834
202.48456731316588
200.9601950635996
201.2411488736834
199.7223072167954
200.9601950635996
202.6079958935481
201.64572894063488
200.84820138602188
200.84820138602188
199.88246546408217
198.9170681465017
199.88246546408217
201.8142710513803
200.84820138602188
199.6246477767713
200.33971149025845
200.33971149025845
198.8793604173143
200.82081565415473
200.09247861926244
198.8793604173143
200.09247861926244
200.09247861926244
200.33971149025845
200.33971149025845
197.9090700296477
199.12307751739877
200.09247861926244
198.15398053029367
198.15398053029367
199.37151250868314
200.09247861926244
198.8793604173143
199.12307751739877
196.468827043885
196.69773765857096
196.69773765857096
194.03092537015846
196.69773765857096
197.909070029